In [115]:
import requests
from bs4 import BeautifulSoup
from difflib import get_close_matches
from random import choice, shuffle
from collections import defaultdict
from gensim import corpora
from gensim import similarities
from gensim import models
import nltk
import json
import pickle

nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/beelzebruno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:

def page_rank(query, p_list):
    rk = 0
    corpus = ' '.join(p_list)
    for token in query.split():
        if token in corpus:
            rk += 1
    # return len(get_close_matches(query, p_list, n=10))
    if query in corpus:
        rk += 1
    return rk

    

def get_result(query, values, lsi):
    # create a bag of words from query input
    vec_bow = gensim_dictionary.doc2bow(query.lower().split())
    vec_lsi = lsi[vec_bow]


    # transforming corpus to LSI space and index it
    index = similarities.MatrixSimilarity(lsi[gensim_corpus])


    # Perform a similarity query against the corpus
    simil = index[vec_lsi]  
    simil = sorted(list(enumerate(simil)), key=lambda item: -item[1])


    # topic1, topic2 = simil
    # result = {
    #     translate[topic1[0]]: topic1[1],
    #     translate[topic2[0]]: topic2[1]
    # }
    res = {}
    for idx, perc in simil:
        perc *= 100
        res[values[idx][0]] = perc
    return res  #result


In [111]:
cached = get_cache()
urls = list(cached.keys())
if not urls:
    urls = ['https://g1.globo.com/']
else:
    shuffle(urls)

visited = set()

c = 0
MAX = 15
query = 'frente fria chegando'


while urls:
    url = urls.pop(0)

    if url.endswith('/'):
        url = url[:-1]

    if url in visited: continue
    
        
    r = requests.get(url)
    c += 1

    visited.add(url)
    print(f'URL: {url} | {r.status_code} | V: {len(visited)} | U: {len(urls)}')

    if r.status_code != 200: continue
    
    soup = BeautifulSoup(r.text, 'html.parser')
    for found_url in soup.find_all('a'):
        x = found_url.attrs.get('href', '')
        if not x.startswith('http') or x in visited: continue
        urls.append(x)

    if url in cached: continue
    
    texts = []
    for t in soup.find_all('p')+soup.find_all('span'):
        tx = t.text.lower().replace('\t', '').replace('\n', ' ').strip()
        if len(tx) < 2:
            continue
        for token in tx.split():
            if token in stopwords.words():
                continue
            texts.append(token)

    # texts = [[word for word in txt.split() if word not in stopwords.words()] for txt in texts]
    doc = ' '.join(texts).strip()
    cached[url] = doc

    if c >= MAX:
        break

store(cached)

URL: https://g1.globo.com | 200 | V: 1 | U: 0
URL: https://g1.globo.com/politica/noticia/2023/08/24/cpi-dos-atos-golpistas-tem-bate-boca-entre-deputados-laura-carneiro-e-marco-feliciano.ghtml | 200 | V: 2 | U: 656
URL: https://g1.globo.com/politica/noticia/2023/08/24/pessoa-abjeta-e-misogina-mentirosa-contumaz-eliziane-gama-e-marco-feliciano-trocam-ofensas-na-cpi.ghtml | 200 | V: 3 | U: 1328
URL: https://g1.globo.com/df/distrito-federal/noticia/2023/08/24/cpi-dos-atos-antidemocraticos-mauro-cid-diz-que-recebimento-de-presentes-estava-entre-suas-funcoes.ghtml | 200 | V: 4 | U: 1999
URL: https://g1.globo.com/df/distrito-federal/noticia/2023/08/24/idade-cid-pergunta.ghtml | 200 | V: 5 | U: 2672
URL: https://g1.globo.com/economia/imposto-de-renda/noticia/2023/08/24/imposto-de-renda-2023-consulta-ao-4o-lote-da-restituicao-comeca-nesta-quinta-feira-saiba-como-fazer.ghtml | 200 | V: 6 | U: 3343
URL: https://g1.globo.com/politica/noticia/2023/08/24/senado-aprova-mp-que-reajusta-salario-minimo-

In [120]:
reserve_unseen_urls(urls)

In [124]:
values = [(u, t) for u, t in cached.items()]
txts = [[word for word in document[1].split()] for document in values]

frequency = defaultdict(int)


for text in txts:
    for token in text:
        frequency[token] += 1
txts = [[token for token in text if frequency[token] > 1] for text in txts]


gensim_dictionary = corpora.Dictionary(txts)
gensim_corpus = [gensim_dictionary.doc2bow(text) for text in txts]

lsi = models.LsiModel(gensim_corpus, id2word=gensim_dictionary, num_topics=len(values))

In [125]:
get_result(query, values, lsi)

{'https://g1.globo.com': 0.0,
 'https://g1.globo.com/politica/noticia/2023/08/24/cpi-dos-atos-golpistas-tem-bate-boca-entre-deputados-laura-carneiro-e-marco-feliciano.ghtml': 0.0,
 'https://g1.globo.com/politica/noticia/2023/08/24/pessoa-abjeta-e-misogina-mentirosa-contumaz-eliziane-gama-e-marco-feliciano-trocam-ofensas-na-cpi.ghtml': 0.0,
 'https://g1.globo.com/df/distrito-federal/noticia/2023/08/24/cpi-dos-atos-antidemocraticos-mauro-cid-diz-que-recebimento-de-presentes-estava-entre-suas-funcoes.ghtml': 0.0,
 'https://g1.globo.com/df/distrito-federal/noticia/2023/08/24/idade-cid-pergunta.ghtml': 0.0,
 'https://g1.globo.com/economia/imposto-de-renda/noticia/2023/08/24/imposto-de-renda-2023-consulta-ao-4o-lote-da-restituicao-comeca-nesta-quinta-feira-saiba-como-fazer.ghtml': 0.0,
 'https://g1.globo.com/politica/noticia/2023/08/24/senado-aprova-mp-que-reajusta-salario-minimo-e-amplia-faixa-de-isencao-do-ir.ghtml': 0.0,
 'https://g1.globo.com/mundo/noticia/2023/08/24/putin-rompe-silencio

In [119]:
def store(data):
    try:
        with open('docs/cache.json', 'r+') as f:
            cached = json.load(f)
            cached.update(data)
            json.dump(data, f)
    except:
        with open('docs/cache.json', 'w') as f:
            json.dump(data, f)


def get_cache():
    try:
        with open('docs/cache.json', 'r') as f:
            data = json.load(f)
            return data
    except:
        return {}


def reserve_unseen_urls(data):
    with open('docs/unseen_urls.bin', 'wb') as f:
        pickle.dump(data, f)


In [102]:
store(cached)